# ACP / EIA Combined table

In [2]:
import pandas as pd

In [6]:
from google.cloud import bigquery

def get_bigquery_table_version(dataset_id, table_name, project_id="dbcp-dev-350818"):
    """
    Get the data version of a BigQuery table.

    The dbcp.commands.publish script generates a version number for each data release
    and adds it as a label to the BQ tables.

    Args:
        dataset_id: the BQ dataset ID
        table_name: the name of the table
        project_id: the GCP project id

    Return:
        the current DBCP version number of the requested table
    """
    client = bigquery.Client()

    table_ref = f"{project_id}.{dataset_id}.{table_name}"
    table = client.get_table(table_ref)  # Fetch table metadata

    labels = table.labels  # Get the labels dictionary
    return labels["version"]

## Get old data from published archives

In [13]:
from dbcp.extract.helpers import cache_gcs_archive_file_locally

table_name = "county_concrete_mw"
version = get_bigquery_table_version("data_mart_dev", table_name)
uri = f"gs://dgm-outputs/{version}/data_mart/{table_name}.parquet"
data_cache = "/app/data/gcp_outputs"

county_concrete_mw_path = cache_gcs_archive_file_locally(uri, data_cache)
county_concrete_mw_old = pd.read_parquet(county_concrete_mw_path)

In [14]:
county_concrete_mw_old.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 455 entries, 0 to 454
Data columns (total 9 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   state_id_fips                    455 non-null    string 
 1   county_id_fips                   455 non-null    string 
 2   state                            455 non-null    string 
 3   county                           455 non-null    string 
 4   iso_region                       455 non-null    string 
 5   resource_clean                   455 non-null    string 
 6   capacity_under_construction_mw   295 non-null    float64
 7   capacity_awaiting_permitting_mw  202 non-null    float64
 8   capacity_total_proposed_mw       455 non-null    float64
dtypes: float64(3), string(6)
memory usage: 32.1 KB


## Get new data from local parquet

In [22]:
county_concrete_mw_new = pd.read_parquet(
    "../../../data/output/data_mart/county_concrete_mw.parquet")
county_concrete_mw_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 463 entries, 0 to 462
Data columns (total 9 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   state_id_fips                    463 non-null    string 
 1   county_id_fips                   463 non-null    string 
 2   state                            463 non-null    string 
 3   county                           463 non-null    string 
 4   iso_region                       463 non-null    string 
 5   resource_clean                   463 non-null    string 
 6   capacity_under_construction_mw   280 non-null    float64
 7   capacity_awaiting_permitting_mw  228 non-null    float64
 8   capacity_total_proposed_mw       463 non-null    float64
dtypes: float64(3), string(6)
memory usage: 32.7 KB


## Sanity check: are old and new tables different

In [24]:
capacity_by_iso_region_new.equals(capacity_by_iso_region_old)

False

In [26]:
# pd.testing.assert_frame_equal(
#     capacity_by_iso_region_new,
#     capacity_by_iso_region_old
# )

## Simplify ISO region

In [33]:
GS_REGIONS = ("MISO", "NYISO", "ISONE", "PJM", "ERCOT", "SPP", "CAISO")

In [35]:
county_concrete_mw_new["iso_region_clean"] = county_concrete_mw_new["iso_region"].mask(
    ~county_concrete_mw_new["iso_region"].isin(GS_REGIONS), other="NON-ISO")

county_concrete_mw_old["iso_region_clean"] = county_concrete_mw_old["iso_region"].mask(
    ~county_concrete_mw_old["iso_region"].isin(GS_REGIONS), other="NON-ISO")

## % change in capacity by ISO region

In [36]:
capacity_by_iso_region_new = county_concrete_mw_new.groupby("iso_region_clean").sum()[
['capacity_under_construction_mw', 'capacity_awaiting_permitting_mw', 'capacity_total_proposed_mw']]

capacity_by_iso_region_old = county_concrete_mw_old.groupby("iso_region_clean").sum()[
['capacity_under_construction_mw', 'capacity_awaiting_permitting_mw', 'capacity_total_proposed_mw']]

In [37]:
capacity_by_iso_region_pct_change = (capacity_by_iso_region_new - capacity_by_iso_region_old) / capacity_by_iso_region_old

In [40]:
capacity_by_iso_region_pct_change.sort_values(by="capacity_total_proposed_mw")

,capacity_under_construction_mw,capacity_awaiting_permitting_mw,capacity_total_proposed_mw
iso_region_clean,,,
MISO,-0.403034,-0.128560,-0.290079
PJM,-0.267633,-0.130375,-0.187237
SPP,0.128253,-0.171739,-0.036461
ISONE,-0.574351,1.355116,-0.034970
CAISO,-0.164055,0.567367,0.083519
NYISO,0.982113,0.023332,0.237549
ERCOT,0.201572,1.508437,0.560390
NON-ISO,0.065555,1.547812,0.637978


## Drill down into individual projects

In order to understand these numbers better, look at individual projects in the old and new data

In [42]:
# Note that this requires comparing old and new results of the query `eia860m_current.sql`
# Until this has been added as an intermediate table, it can be saved as a parquet manually and compared

eia_old = pd.read_parquet("eia860m_current_old.parquet")
eia_new = pd.read_parquet("eia860m_current_new.parquet")

In [159]:
eia_new.head()

,report_date,plant_id_eia,plant_name_eia,utility_id_eia,utility_name_eia,generator_id,capacity_mw,state_id_fips,county_id_fips,state,...,planned_derate_date,planned_generator_retirement_date,planned_net_summer_capacity_derate_mw,planned_net_summer_capacity_uprate_mw,planned_uprate_date,technology_description,raw_state,raw_county,iso_region_clean,unique_id
0,2024-12-01,229,Cow Creek,14328,Pacific Gas & Electric Co.,1,0.7,06,06089,California,...,NaT,NaT,NaN,NaN,NaT,Conventional Hydroelectric,CA,Shasta,CAISO,229_1
1,2024-12-01,229,Cow Creek,14328,Pacific Gas & Electric Co.,2,0.7,06,06089,California,...,NaT,NaT,NaN,NaN,NaT,Conventional Hydroelectric,CA,Shasta,CAISO,229_2
2,2024-12-01,253,Kilarc,14328,Pacific Gas & Electric Co.,1,1.5,06,06089,California,...,NaT,NaT,NaN,NaN,NaT,Conventional Hydroelectric,CA,Shasta,CAISO,253_1
3,2024-12-01,594,Indian River Generating Station,9332,Indian River Operations Inc,4,445.5,10,10005,Delaware,...,NaT,2025-02-01,NaN,NaN,NaT,Conventional Steam Coal,DE,Sussex,PJM,594_4
4,2024-12-01,645,Big Bend,18454,Tampa Electric Co,ST4,486.0,12,12057,Florida,...,NaT,NaT,37.0,NaN,NaT,Conventional Steam Coal,FL,Hillsborough,NON-ISO,645_ST4


In [47]:
eia_old['iso_region_clean'] = eia_old['iso_region'].mask(~eia_old["iso_region"].isin(GS_REGIONS), other="NON-ISO")
eia_new['iso_region_clean'] = eia_new['iso_region'].mask(~eia_new["iso_region"].isin(GS_REGIONS), other="NON-ISO")

In [50]:
eia_new.columns

Index(['report_date', 'plant_id_eia', 'plant_name_eia', 'utility_id_eia',
       'utility_name_eia', 'generator_id', 'capacity_mw', 'state_id_fips',
       'county_id_fips', 'state', 'county', 'iso_region',
       'current_planned_generator_operating_date', 'energy_source_code_1',
       'prime_mover_code', 'energy_storage_capacity_mwh',
       'fuel_type_code_pudl', 'generator_retirement_date', 'latitude',
       'longitude', 'operational_status_code', 'operational_status_category',
       'raw_operational_status_code', 'planned_derate_date',
       'planned_generator_retirement_date',
       'planned_net_summer_capacity_derate_mw',
       'planned_net_summer_capacity_uprate_mw', 'planned_uprate_date',
       'technology_description', 'raw_state', 'raw_county',
       'iso_region_clean'],
      dtype='object')

In [95]:
eia_new.groupby(
    ["operational_status_code", "raw_operational_status_code", "operational_status_category"]
).plant_id_eia.count()

operational_status_code  raw_operational_status_code  operational_status_category
1                        P                            proposed                        75
2                        L                            proposed                        49
3                        T                            proposed                        31
4                        U                            proposed                       120
5                        V                            proposed                       133
6                        TS                           proposed                        81
7                        OA                           existing                        16
                         OP                           existing                       600
                         OS                           existing                         6
                         SB                           existing                         3
8                        RE 

In [79]:
# Combination of plant ID and generator ID is unique
eia_new.groupby(["plant_id_eia", "generator_id"]).count()["plant_name_eia"].max()

1

In [153]:
eia_plant_region_status_capacity_new = eia_new.groupby(
    ['plant_id_eia', 'iso_region_clean', 'operational_status_category'], dropna=False
).agg({"capacity_mw": sum, "plant_id_eia": 'nunique'}).rename(columns={"plant_id_eia": "plant_count"}).unstack()
eia_plant_region_status_capacity_old = eia_old.groupby(
    ['plant_id_eia', 'iso_region_clean', 'operational_status_category'], dropna=False
).agg({"capacity_mw": sum, "plant_id_eia": 'nunique'}).rename(columns={"plant_id_eia": "plant_count"}).unstack()

In [156]:
eia_plant_region_status_capacity_combined = eia_plant_region_status_capacity_old.join(
    eia_plant_region_status_capacity_new, 
    how="outer",
    lsuffix="_old",
    rsuffix="_new",
)

In [157]:
eia_plant_region_status_capacity_combined

capacity_mw_old                   \
operational_status_category          existing proposed retired   
plant_id_eia iso_region_clean                                    
229          CAISO                        NaN      NaN     NaN   
253          CAISO                        NaN      NaN     NaN   
594          PJM                          NaN      NaN     NaN   
645          NON-ISO                      NaN      NaN     NaN   
676          NON-ISO                      NaN      NaN     NaN   
...                                       ...      ...     ...   
68487        NON-ISO                      NaN      NaN     NaN   
68488        NON-ISO                      NaN      NaN     NaN   
68490        ISONE                        NaN      NaN     NaN   
68491        ISONE                        NaN      NaN     NaN   
68493        ISONE                        NaN      NaN     NaN   

                              plant_count_old                   \
operational_status_category          existing proposed retired   
plant_id_eia iso_region_clean                                    
229          CAISO                        NaN      NaN     NaN   
253          CAISO                        NaN      NaN     NaN   
594          PJM                          NaN      NaN     NaN   
645          NON-ISO                      NaN      NaN     NaN   
676          NON-ISO                      NaN      NaN     NaN   
...                                       ...      ...     ...   
68487        NON-ISO                      NaN      NaN     NaN   
68488        NON-ISO                      NaN      NaN     NaN   
68490        ISONE                        NaN      NaN     NaN   
68491        ISONE                        NaN      NaN     NaN   
68493        ISONE                        NaN      NaN     NaN   

                              capacity_mw_new                   \
operational_status_category          existing proposed retired   
plant_id_eia iso_region_clean                                    
229          CAISO                        NaN      NaN     1.4   
253          CAISO                        NaN      NaN     1.5   
594          PJM                        445.5      NaN     NaN   
645          NON-ISO                    486.0      NaN     NaN   
676          NON-ISO                     60.0      NaN     NaN   
...                                       ...      ...     ...   
68487        NON-ISO                      NaN    125.0     NaN   
68488        NON-ISO                      NaN     80.0     NaN   
68490        ISONE                        NaN    200.0     NaN   
68491        ISONE                        NaN    250.0     NaN   
68493        ISONE                       20.4      NaN     NaN   

                              plant_count_new                   
operational_status_category          existing proposed retired  
plant_id_eia iso_region_clean                                   
229          CAISO                        NaN      NaN     1.0  
253          CAISO                        NaN      NaN     1.0  
594          PJM                          1.0      NaN     NaN  
645          NON-ISO                      1.0      NaN     NaN  
676          NON-ISO                      1.0      NaN     NaN  
...                                       ...      ...     ...  
68487        NON-ISO                      NaN      1.0     NaN  
68488        NON-ISO                      NaN      1.0     NaN  
68490        ISONE                        NaN      1.0     NaN  
68491        ISONE                        NaN      1.0     NaN  
68493        ISONE                        1.0      NaN     NaN  

[1208 rows x 12 columns]

In [146]:
eia_plant_region_status_capacity_combined.groupby(["iso_region_clean", "operational_status_category"]).sum().assign(
    plant_count_pct_change=lambda df: (df.plant_count_new - df.plant_count_old) / df.plant_count_old,
    capacity_mw_pct_change=lambda df: (df.capacity_mw_new - df.capacity_mw_old) / df.capacity_mw_old,
)

capacity_mw_old  \
iso_region_clean operational_status_category                    
CAISO            existing                          956.500000   
                 proposed                         2388.400002   
                 retired                             0.000000   
ERCOT            existing                         5840.900045   
                 proposed                         8317.899995   
                 retired                           399.699997   
ISONE            existing                          180.700000   
                 proposed                          925.800000   
MISO             existing                         1126.700004   
                 proposed                         6303.399992   
                 retired                             0.000000   
NON-ISO          existing                         6129.300010   
                 proposed                         8761.500015   
                 retired                           272.000000   
NYISO            existing                           49.100000   
                 proposed                          325.900000   
                 retired                             1.800000   
PJM              existing                          513.800001   
                 proposed                         5302.399998   
                 retired                            37.900000   

                                              plant_count_old  \
iso_region_clean operational_status_category                    
CAISO            existing                                33.0   
                 proposed                                32.0   
                 retired                                  0.0   
ERCOT            existing                                15.0   
                 proposed                                49.0   
                 retired                                  2.0   
ISONE            existing                                34.0   
                 proposed                                21.0   
MISO             existing                                20.0   
                 proposed                                55.0   
                 retired                                  0.0   
NON-ISO          existing                                44.0   
                 proposed                                91.0   
                 retired                                  1.0   
NYISO            existing                                16.0   
                 proposed                                20.0   
                 retired                                  1.0   
PJM              existing                                29.0   
                 proposed                                55.0   
                 retired                                  2.0   

                                              capacity_mw_new  \
iso_region_clean operational_status_category                    
CAISO            existing                         3925.500012   
                 proposed                         5949.900004   
                 retired                             9.600000   
ERCOT            existing                         4074.900002   
                 proposed                        19191.499981   
                 retired                            19.400000   
ISONE            existing                          216.599999   
                 proposed                          973.199999   
MISO             existing                        18904.099933   
                 proposed                         5205.100012   
                 retired                          1582.699999   
NON-ISO          existing                         9799.300017   
                 proposed                        14124.699980   
                 retired                           586.099998   
NYISO            existing                          724.699999   
                 proposed                         1472.500001   
              

In [164]:
eia_combined = pd.merge(
    eia_old, 
    eia_new[
        ['report_date', 
         'plant_id_eia',
         'plant_name_eia',
         'generator_id',
         'utility_id_eia',
         'utility_name_eia',
         'iso_region_clean',
         'capacity_mw', 
         'current_planned_generator_operating_date', 
         'operational_status_code',
         'operational_status_category',
         'raw_operational_status_code',
        ]
    ],
    on=["plant_id_eia", "plant_name_eia", "generator_id",'utility_id_eia', 'utility_name_eia' , "iso_region_clean"],
    how="outer",
    suffixes=("_old", "_new"),
)

#### Check that join did not cause a fan-out

In [165]:
eia_combined.shape

(1712, 39)

In [166]:
eia_new["unique_id"] = eia_new["plant_id_eia"].map(str) + "_" + eia_new["generator_id"].map(str)
eia_old["unique_id"] = eia_old["plant_id_eia"].map(str) + "_" + eia_old["generator_id"].map(str)

In [167]:
len(set(eia_new.unique_id.to_list() + eia_old.unique_id.to_list()))

1698

In [168]:
eia_combined.query("report_date_old.isnull()").iso_region_clean.unique()

array(['CAISO', 'PJM', 'NON-ISO', 'MISO', 'NYISO', 'ISONE', 'ERCOT'],
      dtype=object)

### ERCOT

In [169]:
(
    eia_combined.query("iso_region_clean == 'ERCOT'")
    .groupby(
        ["raw_operational_status_code_old", 
         "operational_status_category_old", 
         "raw_operational_status_code_new",
         "operational_status_category_new",
        ],
        dropna=False
    )
    .agg({"plant_id_eia": "nunique", "capacity_mw_old": sum, "capacity_mw_new": sum})
    .rename(columns={"plant_id_eia": "plant_count"})
    .assign(capacity_pct_change=lambda df: (df.capacity_mw_new - df.capacity_mw_old) / df.capacity_mw_old)
).sort_values(by="capacity_pct_change")

,,,,plant_count,capacity_mw_old,capacity_mw_new,capacity_pct_change
raw_operational_status_code_old,operational_status_category_old,raw_operational_status_code_new,operational_status_category_new,,,,
L,proposed,NaN,NaN,6,360.900003,0.000000,-1.000000
V,proposed,NaN,NaN,5,878.000000,0.000000,-1.000000
OP,existing,NaN,NaN,14,5653.400045,0.000000,-1.000000
U,proposed,NaN,NaN,7,876.499999,0.000000,-1.000000
P,proposed,NaN,NaN,5,960.099998,0.000000,-1.000000
RE,retired,NaN,NaN,2,399.699997,0.000000,-1.000000
T,proposed,NaN,NaN,1,334.299988,0.000000,-1.000000
TS,proposed,NaN,NaN,9,1546.300006,0.000000,-1.000000
V,proposed,V,proposed,3,925.000000,925.000000,0.000000


In [170]:
old_by_status_category = (
    eia_combined.query("iso_region_clean == 'ERCOT'")
    .groupby(
        [
         "operational_status_category_old", 
        ],
        dropna=False
    )
    .agg({"plant_id_eia": "nunique", "capacity_mw_old": sum})
    .rename(columns={"plant_id_eia": "plant_count"})
)
old_by_status_category

,plant_count,capacity_mw_old
operational_status_category_old,,
existing,15,5840.900045
proposed,49,8317.899995
retired,2,399.699997
NaN,91,0.000000


## Focus on ERCOT plants that disappeared

In [171]:
eia_ercot = eia_combined.query("iso_region_clean == 'ERCOT'")

#### % plants ERCOT disappearing

In [172]:
ercot_total_by_status_cat = (
    eia_ercot
    .groupby("operational_status_category_old")
    .agg({"capacity_mw_old": sum, "capacity_mw_new": sum})
)

ercot_disappeared_by_status_cat = (
    eia_ercot.query("operational_status_category_new.isnull()")
    .groupby("operational_status_category_old")
    .agg({"capacity_mw_old": sum, "capacity_mw_new": sum})
)

(ercot_disappeared_by_status_cat - ercot_total_by_status_cat) / ercot_total_by_status_cat

,capacity_mw_old,capacity_mw_new
operational_status_category_old,,
existing,-0.032101,-1.0
proposed,-0.404165,-1.0
retired,0.000000,NaN


In [173]:
(
    eia_combined.query("iso_region_clean == 'ERCOT' and operational_status_code_new.isnull()")
    .groupby(
        ["operational_status_code_old", "raw_operational_status_code_old", "operational_status_category_old"],
        dropna=False
    )
    .agg({"plant_id_eia": "nunique", "capacity_mw_old": sum})
).rename(columns={"plant_id_eia": "plant_count"})

,,,plant_count,capacity_mw_old
operational_status_code_old,raw_operational_status_code_old,operational_status_category_old,,
1.0,P,proposed,5,960.099998
2.0,L,proposed,6,360.900003
3.0,T,proposed,1,334.299988
4.0,U,proposed,7,876.499999
5.0,V,proposed,5,878.000000
6.0,TS,proposed,9,1546.300006
7.0,OP,existing,14,5653.400045
8.0,RE,retired,2,399.699997


In [174]:
eia_combined.query(
    "iso_region_clean == 'ERCOT' " \
    "and operational_status_code_old.isnull() " \
    "and raw_operational_status_code_new == 'OP' " \
)

,report_date_old,plant_id_eia,plant_name_eia,utility_id_eia,utility_name_eia,generator_id,capacity_mw_old,state_id_fips,county_id_fips,state,...,raw_state,raw_county,iso_region_clean,unique_id,report_date_new,capacity_mw_new,current_planned_generator_operating_date_new,operational_status_code_new,operational_status_category_new,raw_operational_status_code_new
776,NaT,7512,Arthur Von Rosenberg,16604,City of San Antonio - (TX),2,NaN,NaN,NaN,NaN,...,NaN,NaN,ERCOT,NaN,2024-12-01,187.500000,NaT,7.0,existing,OP
804,NaT,50815,"Odyssey Energy Altura Cogen, LLC",55879,Odyssey Operating Services,GEN1,NaN,NaN,NaN,NaN,...,NaN,NaN,ERCOT,NaN,2024-12-01,84.900002,NaT,7.0,existing,OP
805,NaT,50815,"Odyssey Energy Altura Cogen, LLC",55879,Odyssey Operating Services,GEN2,NaN,NaN,NaN,NaN,...,NaN,NaN,ERCOT,NaN,2024-12-01,84.900002,NaT,7.0,existing,OP
806,NaT,50815,"Odyssey Energy Altura Cogen, LLC",55879,Odyssey Operating Services,GEN3,NaN,NaN,NaN,NaN,...,NaN,NaN,ERCOT,NaN,2024-12-01,84.900002,NaT,7.0,existing,OP
807,NaT,50815,"Odyssey Energy Altura Cogen, LLC",55879,Odyssey Operating Services,GEN4,NaN,NaN,NaN,NaN,...,NaN,NaN,ERCOT,NaN,2024-12-01,84.900002,NaT,7.0,existing,OP
808,NaT,50815,"Odyssey Energy Altura Cogen, LLC",55879,Odyssey Operating Services,GEN5,NaN,NaN,NaN,NaN,...,NaN,NaN,ERCOT,NaN,2024-12-01,84.900002,NaT,7.0,existing,OP
809,NaT,50815,"Odyssey Energy Altura Cogen, LLC",55879,Odyssey Operating Services,GEN6,NaN,NaN,NaN,NaN,...,NaN,NaN,ERCOT,NaN,2024-12-01,129.199997,NaT,7.0,existing,OP
810,NaT,50815,"Odyssey Energy Altura Cogen, LLC",55879,Odyssey Operating Services,GEN7,NaN,NaN,NaN,NaN,...,NaN,NaN,ERCOT,NaN,2024-12-01,89.900002,NaT,7.0,existing,OP
861,NaT,55091,Midlothian Energy Facility,12501,Midlothian Energy LLC,STK1,NaN,NaN,NaN,NaN,...,NaN,NaN,ERCOT,NaN,2024-12-01,289.000000,NaT,7.0,existing,OP
920,NaT,56754,Goat Wind LP,64644,Goat Wind LLC,1,NaN,NaN,NaN,NaN,...,NaN,NaN,ERCOT,NaN,2024-12-01,80.000000,NaT,7.0,existing,OP


In [175]:
eia_combined.query("iso_region_clean == 'ERCOT' and operational_status_code_new.isnull() and raw_operational_status_code_old == 'OP'")

,report_date_old,plant_id_eia,plant_name_eia,utility_id_eia,utility_name_eia,generator_id,capacity_mw_old,state_id_fips,county_id_fips,state,...,raw_state,raw_county,iso_region_clean,unique_id,report_date_new,capacity_mw_new,current_planned_generator_operating_date_new,operational_status_code_new,operational_status_category_new,raw_operational_status_code_new
16,2024-09-01,3439,Laredo,16604,City of San Antonio - (TX),4,131.800003,48,48479,Texas,...,TX,Webb,ERCOT,3439_4,NaT,NaN,NaT,NaN,NaN,NaN
17,2024-09-01,3439,Laredo,16604,City of San Antonio - (TX),5,131.800003,48,48479,Texas,...,TX,Webb,ERCOT,3439_5,NaT,NaN,NaT,NaN,NaN,NaN
20,2024-09-01,3441,Nueces Bay,16604,City of San Antonio - (TX),7,351.000000,48,48355,Texas,...,TX,Nueces,ERCOT,3441_7,NaT,NaN,NaT,NaN,NaN,NaN
21,2024-09-01,3441,Nueces Bay,16604,City of San Antonio - (TX),8,189.600006,48,48355,Texas,...,TX,Nueces,ERCOT,3441_8,NaT,NaN,NaT,NaN,NaN,NaN
22,2024-09-01,3441,Nueces Bay,16604,City of San Antonio - (TX),9,189.600006,48,48355,Texas,...,TX,Nueces,ERCOT,3441_9,NaT,NaN,NaT,NaN,NaN,NaN
23,2024-09-01,4939,Barney M Davis,16604,City of San Antonio - (TX),1,352.000000,48,48355,Texas,...,TX,Nueces,ERCOT,4939_1,NaT,NaN,NaT,NaN,NaN,NaN
24,2024-09-01,4939,Barney M Davis,16604,City of San Antonio - (TX),2,351.000000,48,48355,Texas,...,TX,Nueces,ERCOT,4939_2,NaT,NaN,NaT,NaN,NaN,NaN
25,2024-09-01,4939,Barney M Davis,16604,City of San Antonio - (TX),3,189.600006,48,48355,Texas,...,TX,Nueces,ERCOT,4939_3,NaT,NaN,NaT,NaN,NaN,NaN
26,2024-09-01,4939,Barney M Davis,16604,City of San Antonio - (TX),4,189.600006,48,48355,Texas,...,TX,Nueces,ERCOT,4939_4,NaT,NaN,NaT,NaN,NaN,NaN
95,2024-09-01,55137,Rio Nogales Power Project,16604,City of San Antonio - (TX),CTG3,189.000000,48,48187,Texas,...,TX,Guadalupe,ERCOT,55137_CTG3,NaT,NaN,NaT,NaN,NaN,NaN


- Compare % change for existing plants to % change in retirement (if existing decreases, expect retirement to increase)
- For all plants make sure status change is sensible (join on table and ID)
- For data integrity, compute % of plants that behave unexpectedly (i.e. disappear from data)
    * On merged table, combine before and after state and group by these "paths" to see how common different changes are
    * Check individual generator capacity changing
    * Generate chart for previous quarterly updates to get historic context

## By State

In [41]:
capacity_by_state_new = county_concrete_mw_new.groupby("state").sum()[
['capacity_under_construction_mw', 'capacity_awaiting_permitting_mw', 'capacity_total_proposed_mw']]

capacity_by_state_old = county_concrete_mw_old.groupby("state").sum()[
['capacity_under_construction_mw', 'capacity_awaiting_permitting_mw', 'capacity_total_proposed_mw']]

capacity_by_state_pct_change = (capacity_by_state_new - capacity_by_state_old) / capacity_by_state_old
capacity_by_state_pct_change.sort_values(by="capacity_total_proposed_mw")

,capacity_under_construction_mw,capacity_awaiting_permitting_mw,capacity_total_proposed_mw
state,,,
Rhode Island,-0.996573,0.000000,-0.983097
Louisiana,-0.865342,-0.571429,-0.749550
North Carolina,-0.973665,0.149715,-0.685760
New Mexico,-0.855556,0.000000,-0.642523
Pennsylvania,0.657201,-0.829352,-0.561816
South Carolina,0.452599,-0.732278,-0.529103
Illinois,-0.905673,2.219531,-0.525004
Idaho,0.327217,-1.000000,-0.513998
Arkansas,-0.195084,-1.000000,-0.508443
